# Fine pruning defense

This notebook will contain code to run the fine prining defense https://link.springer.com/chapter/10.1007/978-3-030-00470-5_13

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16

from kerassurgeon import Surgeon
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels

## Data

Placeholder

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Model

Placeholder

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
batch_size = 128
epochs = 6

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/6
422/422 [==============================] - 2s 4ms/step - loss: 0.3719 - accuracy: 0.8853 - val_loss: 0.0825 - val_accuracy: 0.9773
Epoch 2/6
422/422 [==============================] - 2s 4ms/step - loss: 0.1151 - accuracy: 0.9636 - val_loss: 0.0576 - val_accuracy: 0.9843
Epoch 3/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0862 - accuracy: 0.9736 - val_loss: 0.0469 - val_accuracy: 0.9875
Epoch 4/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0726 - accuracy: 0.9780 - val_loss: 0.0446 - val_accuracy: 0.9888
Epoch 5/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0631 - accuracy: 0.9801 - val_loss: 0.0376 - val_accuracy: 0.9895
Epoch 6/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0564 - accuracy: 0.9827 - val_loss: 0.0357 - val_accuracy: 0.9907


In [4]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
original_acc = score[1]

Test loss: 0.03334033116698265
Test accuracy: 0.9891999959945679


In [5]:
print(model.count_params())
print(model.summary())

34826
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)           

## Pruning

In [6]:
# last_conv, last_conv_bias = model.get_layer('conv2d_1').get_weights()
# last_conv = np.copy(last_conv)
# last_conv_bias = np.copy(last_conv_bias)
# last_conv_flat = np.copy(last_conv).flatten()
# print(last_conv.shape, last_conv_bias.shape, last_conv_flat.shape)

# restore = last_conv_flat.reshape(last_conv.shape)
# assert np.array_equal(last_conv, restore)

In [7]:
# small_idx = np.argpartition(last_conv_flat, last_conv_flat.shape[0] - 2)
# print(small_idx)

In [8]:
# stop_cond = False
# cur_idx = 0

# while not stop_cond:
#     last_conv_flat[small_idx[cur_idx]] = 0.0
#     last_conv_pruned = np.copy(last_conv_flat)
#     last_conv_pruned = last_conv_pruned.reshape(last_conv.shape)

#     new_weights = [last_conv_pruned, last_conv_bias]
#     model.get_layer('conv2d_1').set_weights(new_weights)
    
#     score = model.evaluate(x_test, y_test, verbose=0)
    
#     if cur_idx % 10 == 0:
#         print('Number of weights set to 0:', cur_idx)
#         print("Test loss:", score[0])
#         print("Test accuracy:", score[1])
    
#     cur_idx += 1
#     if original_acc - score[1] > .04:
#         stop_cond = True
#     elif cur_idx == last_conv_flat.shape[0]:
#         stop_cond = True
        
# print('\nFINAL')
# print('Number of weights set to 0:', cur_idx)
# print("Test loss:", score[0])
# print("Test accuracy:", score[1])

In [10]:
stop_cond = False
cur_idx = 0

while not stop_cond:
    target_layer = model.get_layer('conv2d_1')
    apoz = identify.get_apoz(model, target_layer, x_test)
    high_apoz_channels = identify.high_apoz(apoz, "both")
    if len(high_apoz_channels) == 0: continue
    high_apoz_channels = [high_apoz_channels[0]]
    print('Neurons in layer', len(apoz))    
    print('Neurons to prune', high_apoz_channels)
    
    model = delete_channels(model, target_layer, high_apoz_channels)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    score = model.evaluate(x_test, y_test, verbose=0)

    print('Number of neurons pruned:', cur_idx)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    cur_idx += 1
    if original_acc - score[1] > .04:
        stop_cond = True
        
print('\nFINAL')
print('Number of weights set to 0:', cur_idx)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Neurons in layer 64
Neurons to prune [0]
Deleting 1/64 channels from layer: conv2d_1
Number of neurons pruned: 0
Test loss: 0.03523044288158417
Test accuracy: 0.9883999824523926
Neurons in layer 63
Neurons to prune [2]
Deleting 1/63 channels from layer: conv2d_1
Number of neurons pruned: 1
Test loss: 0.03626205027103424
Test accuracy: 0.9882000088691711
Neurons in layer 62
Neurons to prune [8]
Deleting 1/62 channels from layer: conv2d_1
Number of neurons pruned: 2
Test loss: 0.03772997483611107
Test accuracy: 0.9876999855041504
Neurons in layer 61
Neurons to prune [9]
Deleting 1/61 channels from layer: conv2d_1
Number of neurons pruned: 3
Test loss: 0.040567416697740555
Test accuracy: 0.9866999983787537
Neurons in layer 60
Neurons to prune [9]
Deleting 1/60 channels from layer: conv2d_1
Number of neurons pruned: 4
Test loss: 0.04332003369927406
Test accuracy: 0.9865000247955322
Neurons in layer 59
Neurons to prune [18]
Deleting 1/59 channels from layer: conv2d_1
Number of neurons prune

In [11]:
print(model.count_params())
print(model.summary())

22429
Model: "functional_93"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 41)        11849     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple          

In [12]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.1632322072982788
Test accuracy: 0.9459999799728394


In [17]:
model.optimizer.learning_rate.assign(0.0001)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [18]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0859 - accuracy: 0.9740 - val_loss: 0.0413 - val_accuracy: 0.9900
Epoch 2/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0708 - accuracy: 0.9786 - val_loss: 0.0386 - val_accuracy: 0.9907
Epoch 3/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0656 - accuracy: 0.9791 - val_loss: 0.0372 - val_accuracy: 0.9908
Epoch 4/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0637 - accuracy: 0.9801 - val_loss: 0.0363 - val_accuracy: 0.9907
Epoch 5/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0616 - accuracy: 0.9807 - val_loss: 0.0355 - val_accuracy: 0.9905
Epoch 6/6
422/422 [==============================] - 2s 4ms/step - loss: 0.0623 - accuracy: 0.9801 - val_loss: 0.0345 - val_accuracy: 0.9912


In [19]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.034625083208084106
Test accuracy: 0.9890999794006348


In [20]:
print(model.count_params())
print(model.summary())

22429
Model: "functional_93"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 41)        11849     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple          